In [39]:
import pandas as pd
import numpy as np

In [40]:
df = pd.read_csv("data2.csv")
df.columns = [i.replace(" ", "_").lower() for i in df.columns]
df["date_uploaded_date"] = pd.to_datetime(df["date_uploaded_date"])
df = df[df["date_uploaded_date"] >= "2024-10-01"]
df["date_uploaded_month"] = pd.to_datetime(df["date_uploaded_date"]).dt.to_period("M")
df.head()

,date_uploaded_date,country,product_group,registration_date_month,predicted_frr_updated,count,date_uploaded_month
0,2025-09-01,Kenya,SHS Entry-Level,2025-02,0.860431,6241,2025-09
1,2025-09-01,Kenya,SHS Entry-Level,2022-08,0.845198,6377,2025-09
2,2025-09-01,Kenya,SHS with TV Upgrade,2021-08,0.741954,1250,2025-09
3,2025-09-01,Uganda,SHS Entry-Level,2022-05,0.717826,1012,2025-09
4,2025-09-01,Kenya,Lanterns,2025-03,0.876410,4931,2025-09


In [41]:
print(df["date_uploaded_month"].unique())

<PeriodArray>
['2025-09', '2025-08', '2025-07', '2025-06', '2025-05', '2025-04', '2025-03',
 '2025-02', '2025-01', '2024-12', '2024-11', '2024-10']
Length: 12, dtype: period[M]


In [67]:
date_uploaded = df["date_uploaded_month"].unique()[1]
print(date_uploaded)
subset = df[(df["date_uploaded_month"]==date_uploaded) | (df["date_uploaded_month"]==date_uploaded-1)]

2025-08


In [68]:
backtestings = np.array([30, 60, 90, 180, 270, 360, 450, 540, 630, 720], dtype=int)
months = backtestings // 30
current_month = pd.Timestamp.today().to_period("M")

# vectorized subtraction
backtesting_month_current = date_uploaded - (months + 1)
backtesting_month_prev = date_uploaded - (months + 2)

print(backtesting_month_current)

data = {
    "Months": months,
    "backtesting_month_current": backtesting_month_current,
    "backtesting_month_prev": backtesting_month_prev,
    "backtesting_shift": backtestings
}
backtestings_df = pd.DataFrame(data)

[Period('2025-06', 'M') Period('2025-05', 'M') Period('2025-04', 'M')
 Period('2025-01', 'M') Period('2024-10', 'M') Period('2024-07', 'M')
 Period('2024-04', 'M') Period('2024-01', 'M') Period('2023-10', 'M')
 Period('2023-07', 'M')]


In [69]:
backtestings_df

,Months,backtesting_month_current,backtesting_month_prev,backtesting_shift
0,1,2025-06,2025-05,30
1,2,2025-05,2025-04,60
2,3,2025-04,2025-03,90
3,6,2025-01,2024-12,180
4,9,2024-10,2024-09,270
5,12,2024-07,2024-06,360
6,15,2024-04,2024-03,450
7,18,2024-01,2023-12,540
8,21,2023-10,2023-09,630
9,24,2023-07,2023-06,720


In [70]:
pivoted = subset.pivot_table(
    index=["country", "product_group","registration_date_month"],
    columns="date_uploaded_month",
    values=["predicted_frr_updated", "count"]
).reset_index()

pivoted.columns = [
    str(c[0]) if c[1] == "" or pd.isna(c[1]) else f"{c[0]}_{c[1]}"
    for c in pivoted.columns.to_flat_index()
]
pivoted["registration_date_month"] = (
    pd.to_datetime(pivoted["registration_date_month"], format="%Y-%m")
      .dt.to_period("M")
)


In [71]:
pivoted.head()

,country,product_group,registration_date_month,count_2025-07,count_2025-08,predicted_frr_updated_2025-07,predicted_frr_updated_2025-08
0,Kenya,Lanterns,2021-06,4448.0,4477.0,0.923694,0.921999
1,Kenya,Lanterns,2021-07,4582.0,4612.0,0.923757,0.922243
2,Kenya,Lanterns,2021-08,4510.0,4540.0,0.921187,0.919908
3,Kenya,Lanterns,2021-09,4324.0,4353.0,0.920938,0.919863
4,Kenya,Lanterns,2021-10,4505.0,4535.0,0.923296,0.922101


In [72]:
discrepancies = pd.merge(left=pivoted, right=backtestings_df, left_on="registration_date_month", right_on="backtesting_month_current", how="left")
discrepancies["backtesting_shift"] = discrepancies["backtesting_shift"].astype("Int64")
discrepancies = discrepancies[discrepancies.backtesting_shift.isin([360, 450, 540, 630, 720])]

discrepancies["diff"] = (discrepancies["predicted_frr_updated_2025-08"] - discrepancies["predicted_frr_updated_2025-07"]).round(4)*100

In [85]:
discrepancies.head()

,country,product_group,registration_date_month,count_2025-07,count_2025-08,predicted_frr_updated_2025-07,predicted_frr_updated_2025-08,Months,backtesting_month_current,backtesting_month_prev,backtesting_shift,diff
25,Kenya,Lanterns,2023-07,4644.0,4643.0,0.903819,0.901985,24.0,2023-07,2023-06,720,-0.18
28,Kenya,Lanterns,2023-10,4595.0,4594.0,0.879352,0.878713,21.0,2023-10,2023-09,630,-0.06
31,Kenya,Lanterns,2024-01,4768.0,4768.0,0.891745,0.881083,18.0,2024-01,2023-12,540,-1.07
34,Kenya,Lanterns,2024-04,4707.0,4707.0,0.890225,0.872993,15.0,2024-04,2024-03,450,-1.72
37,Kenya,Lanterns,2024-07,4858.0,4858.0,0.880631,0.884665,12.0,2024-07,2024-06,360,0.40


In [80]:
str(date_uploaded)

'2025-08'

In [79]:
date_uploaded-1

Period('2025-07', 'M')

In [81]:
summary_products = (
    discrepancies
    .groupby(["product_group"])
    .agg(
        total_records=("diff", "size"),
        count_average=("count_2025-08", lambda x: int(x.mean())),
        count_median=("count_2025-08", lambda x: int(x.median())),
        mean_diff=("diff", lambda x: x.mean().round(2)),
        discrepancies_75=("diff", lambda x: (x.abs() > 0.75).sum())
    )
    .reset_index()
)

summary_products["disrepancy75_percent"] = summary_products["discrepancies_75"]/summary_products["total_records"]

summary_products

,product_group,total_records,count_average,count_median,mean_diff,discrepancies_75,disrepancy75_percent
0,Lanterns,22,1502,378,-0.40,7,0.318182
1,Phones,21,1077,669,-0.51,10,0.476190
2,SHS Entry-Level,35,1667,779,-0.39,15,0.428571
3,SHS Entry-Level Upgrade,18,164,10,-0.07,5,0.277778
4,SHS with TV,32,456,118,0.12,16,0.500000
5,SHS with TV Upgrade,25,159,87,-0.23,6,0.240000
6,SHS without TV,30,647,526,-0.50,16,0.533333


In [76]:
discrepancies_summary = (
    discrepancies
    .groupby(["country"])
    .agg(
        total_records=("diff", "size"),
        count_average=("count_2025-08", lambda x: int(x.mean())),
        count_median=("count_2025-08", lambda x: int(x.median())),
        mean_diff=("diff", lambda x: x.mean().round(2)),
        discrepancies_75=("diff", lambda x: (x.abs() > 0.75).sum())
    )
    .reset_index()
)

discrepancies_summary

,country,total_records,count_average,count_median,mean_diff,discrepancies_75
0,Kenya,35,2275,1690,-0.48,17
1,Malawi,17,127,91,-0.26,6
2,Nigeria,25,1247,1119,-0.10,1
3,Tanzania,35,838,669,-0.39,18
4,Togo,8,223,260,1.03,7
5,Uganda,32,126,30,-0.98,20
6,Zambia,31,219,98,0.30,6


In [82]:
discrepancies_summary["date_uploaded"] = "2025-08"

In [83]:
discrepancies_summary

,country,total_records,count_average,count_median,mean_diff,discrepancies_75,date_uploaded
0,Kenya,35,2275,1690,-0.48,17,2025-08
1,Malawi,17,127,91,-0.26,6,2025-08
2,Nigeria,25,1247,1119,-0.10,1,2025-08
3,Tanzania,35,838,669,-0.39,18,2025-08
4,Togo,8,223,260,1.03,7,2025-08
5,Uganda,32,126,30,-0.98,20,2025-08
6,Zambia,31,219,98,0.30,6,2025-08


## Rough

In [109]:
pivoted = df.pivot_table(
    index=["country", "product_group", "registration_date_month"],
    columns="date_uploaded_month",
    values=["predicted_frr_updated", "count"]
).reset_index()
pivoted.columns = [
    str(c[0]) if c[1] == "" or pd.isna(c[1]) else f"{c[0]}_{c[1]}"
    for c in pivoted.columns.to_flat_index()
]


In [110]:
pivoted

,country,product_group,registration_date_month,count_2024-10,count_2024-11,count_2024-12,count_2025-01,count_2025-02,count_2025-03,count_2025-04,...,predicted_frr_updated_2024-12,predicted_frr_updated_2025-01,predicted_frr_updated_2025-02,predicted_frr_updated_2025-03,predicted_frr_updated_2025-04,predicted_frr_updated_2025-05,predicted_frr_updated_2025-06,predicted_frr_updated_2025-07,predicted_frr_updated_2025-08,predicted_frr_updated_2025-09
0,Kenya,Lanterns,2021-06,3648.0,3648.0,3648.0,3648.0,3648.0,4448.0,4448.0,...,0.917024,0.917095,0.917100,0.923682,0.923682,0.923681,0.923688,0.923694,0.921999,0.922010
1,Kenya,Lanterns,2021-07,3766.0,3766.0,3766.0,3765.0,3765.0,4565.0,4565.0,...,0.917091,0.917132,0.917132,0.923794,0.923792,0.923758,0.923758,0.923757,0.922243,0.922247
2,Kenya,Lanterns,2021-08,3714.0,3714.0,3714.0,3713.0,3713.0,4457.0,4488.0,...,0.915239,0.915242,0.915242,0.920996,0.921244,0.921187,0.921184,0.921187,0.919908,0.919909
3,Kenya,Lanterns,2021-09,3563.0,3563.0,3563.0,3561.0,3561.0,4302.0,4302.0,...,0.915242,0.915316,0.915312,0.920970,0.920973,0.920939,0.920937,0.920938,0.919863,0.919856
4,Kenya,Lanterns,2021-10,3709.0,3709.0,3709.0,3708.0,3708.0,4483.0,4483.0,...,0.918335,0.918428,0.918434,0.923380,0.923379,0.923303,0.923303,0.923296,0.922101,0.922100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,Zambia,SHS without TV,2025-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.976674,0.979620,0.985045,0.985019,0.987361
1615,Zambia,SHS without TV,2025-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.978405,0.985699,0.983102,0.986009
1616,Zambia,SHS without TV,2025-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.986601,0.986054,0.984654
1617,Zambia,SHS without TV,2025-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.971452,0.979647


In [ ]:
pivoted['registration_date_month'] = (
    pd.to_datetime(pivoted['registration_date_month'], format="%Y-%m")
      .dt.to_period("M")
)

In [112]:
# Extract FRR columns and sort them
frr_cols = [c for c in pivoted.columns if c.startswith("predicted_frr_updated_")]
frr_cols = sorted(frr_cols, key=lambda x: pd.Period(x.split("_")[-1], freq="M"), reverse=True)

# Compute consecutive differences
# for i in range(len(frr_cols) - 1):
#     col_new = f"m{i}-m{i+1}"
#     pivoted[col_new] = pivoted[frr_cols[i]] - pivoted[frr_cols[i+1]]


for i in range(len(frr_cols) - 1):
    # Parse the current month from the column name
    month_str = frr_cols[i].split("_")[-1]   # e.g., "2025-09"
    col_new = f"m{i}_{month_str}"
    pivoted[col_new] = pivoted[frr_cols[i]] - pivoted[frr_cols[i+1]]


In [113]:
pivoted

,country,product_group,registration_date_month,count_2024-10,count_2024-11,count_2024-12,count_2025-01,count_2025-02,count_2025-03,count_2025-04,...,m1_2025-08,m2_2025-07,m3_2025-06,m4_2025-05,m5_2025-04,m6_2025-03,m7_2025-02,m8_2025-01,m9_2024-12,m10_2024-11
0,Kenya,Lanterns,2021-06,3648.0,3648.0,3648.0,3648.0,3648.0,4448.0,4448.0,...,-0.001695,6.070764e-06,6.435429e-06,-4.891734e-07,1.180558e-07,0.006582,4.946513e-06,0.000071,0.000003,-0.000002
1,Kenya,Lanterns,2021-07,3766.0,3766.0,3766.0,3765.0,3765.0,4565.0,4565.0,...,-0.001514,-3.932135e-07,-6.201392e-07,-3.346284e-05,-1.886971e-06,0.006661,9.072526e-08,0.000042,0.000006,0.000005
2,Kenya,Lanterns,2021-08,3714.0,3714.0,3714.0,3713.0,3713.0,4457.0,4488.0,...,-0.001279,3.557356e-06,-2.799340e-06,-5.716433e-05,2.473322e-04,0.005755,-8.983119e-07,0.000003,0.000003,-0.000003
3,Kenya,Lanterns,2021-09,3563.0,3563.0,3563.0,3561.0,3561.0,4302.0,4302.0,...,-0.001076,1.247646e-06,-1.960985e-06,-3.397121e-05,2.940777e-06,0.005658,-4.480084e-06,0.000074,0.000004,0.000011
4,Kenya,Lanterns,2021-10,3709.0,3709.0,3709.0,3708.0,3708.0,4483.0,4483.0,...,-0.001195,-6.780153e-06,-6.272514e-07,-7.554682e-05,-1.442008e-06,0.004947,6.062251e-06,0.000092,-0.000005,-0.000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,Zambia,SHS without TV,2025-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.000026,5.424773e-03,2.945857e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1615,Zambia,SHS without TV,2025-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.002597,7.293965e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1616,Zambia,SHS without TV,2025-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.000547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1617,Zambia,SHS without TV,2025-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
backtestings = np.array([30, 60, 90, 180, 270, 360, 450, 540, 630, 720], dtype=int)
months = backtestings // 30
current_month = pd.Timestamp.today().to_period("M")

In [115]:
# vectorized subtraction
backtesting_month_current = current_month - (months + 1)
backtesting_month_prev = current_month - (months + 2)

print(backtesting_month_current)

data = {
    "Months": months,
    "backtesting_month_current": backtesting_month_current,
    "backtesting_month_prev": backtesting_month_prev,
    "backtesting_shift": backtestings
}
backtestings_df = pd.DataFrame(data)

[Period('2025-07', 'M') Period('2025-06', 'M') Period('2025-05', 'M')
 Period('2025-02', 'M') Period('2024-11', 'M') Period('2024-08', 'M')
 Period('2024-05', 'M') Period('2024-02', 'M') Period('2023-11', 'M')
 Period('2023-08', 'M')]


In [116]:
pivoted.columns

Index(['country', 'product_group', 'registration_date_month', 'count_2024-10',
       'count_2024-11', 'count_2024-12', 'count_2025-01', 'count_2025-02',
       'count_2025-03', 'count_2025-04', 'count_2025-05', 'count_2025-06',
       'count_2025-07', 'count_2025-08', 'count_2025-09',
       'predicted_frr_updated_2024-10', 'predicted_frr_updated_2024-11',
       'predicted_frr_updated_2024-12', 'predicted_frr_updated_2025-01',
       'predicted_frr_updated_2025-02', 'predicted_frr_updated_2025-03',
       'predicted_frr_updated_2025-04', 'predicted_frr_updated_2025-05',
       'predicted_frr_updated_2025-06', 'predicted_frr_updated_2025-07',
       'predicted_frr_updated_2025-08', 'predicted_frr_updated_2025-09',
       'm0_2025-09', 'm1_2025-08', 'm2_2025-07', 'm3_2025-06', 'm4_2025-05',
       'm5_2025-04', 'm6_2025-03', 'm7_2025-02', 'm8_2025-01', 'm9_2024-12',
       'm10_2024-11'],
      dtype='object')

In [117]:
discrepancies = pd.merge(left=pivoted, right=backtestings_df, left_on='registration_date_month', right_on="backtesting_month_current", how="left")
discrepancies["backtesting_shift"] = discrepancies["backtesting_shift"].astype("Int64")

In [118]:
discrepancies

,country,product_group,registration_date_month,count_2024-10,count_2024-11,count_2024-12,count_2025-01,count_2025-02,count_2025-03,count_2025-04,...,m5_2025-04,m6_2025-03,m7_2025-02,m8_2025-01,m9_2024-12,m10_2024-11,Months,backtesting_month_current,backtesting_month_prev,backtesting_shift
0,Kenya,Lanterns,2021-06,3648.0,3648.0,3648.0,3648.0,3648.0,4448.0,4448.0,...,1.180558e-07,0.006582,4.946513e-06,0.000071,0.000003,-0.000002,NaN,NaT,NaT,<NA>
1,Kenya,Lanterns,2021-07,3766.0,3766.0,3766.0,3765.0,3765.0,4565.0,4565.0,...,-1.886971e-06,0.006661,9.072526e-08,0.000042,0.000006,0.000005,NaN,NaT,NaT,<NA>
2,Kenya,Lanterns,2021-08,3714.0,3714.0,3714.0,3713.0,3713.0,4457.0,4488.0,...,2.473322e-04,0.005755,-8.983119e-07,0.000003,0.000003,-0.000003,NaN,NaT,NaT,<NA>
3,Kenya,Lanterns,2021-09,3563.0,3563.0,3563.0,3561.0,3561.0,4302.0,4302.0,...,2.940777e-06,0.005658,-4.480084e-06,0.000074,0.000004,0.000011,NaN,NaT,NaT,<NA>
4,Kenya,Lanterns,2021-10,3709.0,3709.0,3709.0,3708.0,3708.0,4483.0,4483.0,...,-1.442008e-06,0.004947,6.062251e-06,0.000092,-0.000005,-0.000005,NaN,NaT,NaT,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,Zambia,SHS without TV,2025-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,<NA>
1615,Zambia,SHS without TV,2025-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,<NA>
1616,Zambia,SHS without TV,2025-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2025-05,2025-04,90
1617,Zambia,SHS without TV,2025-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2025-06,2025-05,60


In [119]:
discrepancies_df = discrepancies[discrepancies.backtesting_shift.isin([360, 450, 540, 630, 720])]

In [120]:
discrepancies_df[(discrepancies_df.product_group=="Lanterns") & (discrepancies_df.country=="Kenya")].T.head(45)

,26,29,32,35,38
country,Kenya,Kenya,Kenya,Kenya,Kenya
product_group,Lanterns,Lanterns,Lanterns,Lanterns,Lanterns
registration_date_month,2023-08,2023-11,2024-02,2024-05,2024-08
count_2024-10,3889.0,3703.0,3901.0,4004.0,4068.0
count_2024-11,3889.0,3703.0,3901.0,4004.0,4069.0
count_2024-12,3889.0,3703.0,3901.0,4004.0,4069.0
count_2025-01,3889.0,3703.0,3901.0,4003.0,4068.0
count_2025-02,3889.0,3703.0,3901.0,4003.0,4068.0
count_2025-03,4640.0,4432.0,4639.0,4748.0,4832.0
count_2025-04,4640.0,4432.0,4639.0,4748.0,4832.0


In [121]:
discrepancies_summary = (
    discrepancies_df
    .groupby(["product_group"])
    .agg(
        total_records=("m0_2025-09", "size"),
        count_median=("count_2025-09", lambda x: int(x.median())),
        mean_diff=("m0_2025-09", lambda x: x.mean().round(2)),
        discrepancies_75=("m0_2025-09", lambda x: (x.abs()*100 > 0.75).sum())
    )
    .reset_index()
)

discrepancies_summary

,product_group,total_records,count_median,mean_diff,discrepancies_75
0,Lanterns,22,393,-0.00,8
1,Phones,21,1030,-0.01,7
2,SHS Entry-Level,35,772,-0.00,15
3,SHS Entry-Level Upgrade,17,13,0.00,6
4,SHS with TV,32,134,-0.00,16
5,SHS with TV Upgrade,25,67,-0.00,6
6,SHS without TV,31,290,-0.00,13


In [122]:
discrepancies_summary = (
    discrepancies_df
    .groupby(["country"])
    .agg(
        total_records=("m0_2025-09", "size"),
        count_median=("count_2025-09", lambda x: int(x.median())),
        mean_diff=("m0_2025-09", lambda x: x.mean().round(2)),
        discrepancies_75=("m0_2025-09", lambda x: (x.abs()*100 > 0.75).sum())
    )
    .reset_index()
)

discrepancies_summary

,country,total_records,count_median,mean_diff,discrepancies_75
0,Kenya,35,1802,-0.00,14
1,Malawi,17,84,-0.00,2
2,Nigeria,25,1267,-0.00,2
3,Tanzania,35,539,-0.01,18
4,Togo,9,238,0.00,8
5,Uganda,31,34,-0.01,23
6,Zambia,31,98,-0.00,4


In [123]:
discrepancies_summary = (
    discrepancies_df
    .groupby(["product_group"])
    .agg(
        total_records=("m1_2025-08", "size"),
        count_median=("count_2025-08", lambda x: int(x.median())),
        mean_diff=("m1_2025-08", lambda x: x.mean().round(2)),
        discrepancies_75=("m1_2025-08", lambda x: (x.abs()*100 > 0.75).sum())
    )
    .reset_index()
)

discrepancies_summary

,product_group,total_records,count_median,mean_diff,discrepancies_75
0,Lanterns,22,393,0.0,0
1,Phones,21,1030,0.0,0
2,SHS Entry-Level,35,772,-0.0,0
3,SHS Entry-Level Upgrade,17,13,-0.0,1
4,SHS with TV,32,132,0.0,1
5,SHS with TV Upgrade,25,66,-0.0,0
6,SHS without TV,31,290,0.0,0


In [124]:
discrepancies_summary = (
    discrepancies_df
    .groupby(["product_group"])
    .agg(
        total_records=("m2_2025-07", "size"),
        count_median=("count_2025-07", lambda x: int(x.median())),
        mean_diff=("m2_2025-07", lambda x: x.mean().round(2)),
        discrepancies_75=("m2_2025-07", lambda x: (x.abs()*100 > 0.75).sum())
    )
    .reset_index()
)

discrepancies_summary

,product_group,total_records,count_median,mean_diff,discrepancies_75
0,Lanterns,22,393,-0.0,0
1,Phones,21,1030,-0.0,0
2,SHS Entry-Level,35,772,-0.0,0
3,SHS Entry-Level Upgrade,17,13,-0.0,0
4,SHS with TV,32,132,0.0,1
5,SHS with TV Upgrade,25,67,0.0,2
6,SHS without TV,31,290,0.0,0


In [125]:
discrepancies_summary = (
    discrepancies_df
    .groupby(["product_group"])
    .agg(
        total_records=("m3_2025-06", "size"),
        count_median=("count_2025-06", lambda x: int(x.median())),
        mean_diff=("m3_2025-06", lambda x: x.mean().round(2)),
        discrepancies_75=("m3_2025-06", lambda x: (x.abs()*100 > 0.75).sum())
    )
    .reset_index()
)

discrepancies_summary

,product_group,total_records,count_median,mean_diff,discrepancies_75
0,Lanterns,22,393,-0.00,9
1,Phones,21,1030,-0.01,9
2,SHS Entry-Level,35,772,0.00,12
3,SHS Entry-Level Upgrade,17,13,-0.00,5
4,SHS with TV,32,132,0.00,11
5,SHS with TV Upgrade,25,67,0.00,5
6,SHS without TV,31,290,-0.00,15


In [126]:
delta_cols = [c for c in discrepancies_df.columns if c.startswith("m")]
count_cols = [c for c in discrepancies_df.columns if c.startswith("count_")]

In [127]:
agg_dict = {"total_records": ("country", "size")}  # total_records can be any col

# For each count column, add median
# for col in count_cols:
#     agg_dict[f"median_{col}"] = (col, lambda x: int(x.median()))

# For each delta column, add mean and 75% discrepancies
for col in delta_cols:
    # agg_dict[f"mean_{col}"] = (col, lambda x: x.mean().round(2))
    agg_dict[f"discrepancies75_{col}"] = (col, lambda x: (x.abs()*100 > 1).sum())


In [128]:
discrepancies_summary = (
    discrepancies_df
    .groupby("country")
    .agg(**agg_dict)
    .reset_index()
)
discrepancies_summary

,country,total_records,discrepancies75_m0_2025-09,discrepancies75_m1_2025-08,discrepancies75_m2_2025-07,discrepancies75_m3_2025-06,discrepancies75_m4_2025-05,discrepancies75_m5_2025-04,discrepancies75_m6_2025-03,discrepancies75_m7_2025-02,discrepancies75_m8_2025-01,discrepancies75_m9_2024-12,discrepancies75_m10_2024-11
0,Kenya,35,10,0,0,9,1,0,12,0,3,7,8
1,Malawi,17,1,0,0,2,0,0,2,0,2,5,2
2,Nigeria,25,1,0,0,1,0,0,3,0,0,3,3
3,Tanzania,35,11,0,0,8,0,1,15,0,5,8,8
4,Togo,9,7,0,1,5,2,0,7,1,1,7,3
5,Uganda,31,20,0,0,18,1,0,18,1,1,23,3
6,Zambia,31,1,0,0,6,0,0,11,1,1,10,5


In [129]:
discrepancies_summary.to_csv("country.csv")

In [130]:
discrepancies_summary = (
    discrepancies_df
    .groupby("product_group")
    .agg(**agg_dict)
    .reset_index()
)
discrepancies_summary


,product_group,total_records,discrepancies75_m0_2025-09,discrepancies75_m1_2025-08,discrepancies75_m2_2025-07,discrepancies75_m3_2025-06,discrepancies75_m4_2025-05,discrepancies75_m5_2025-04,discrepancies75_m6_2025-03,discrepancies75_m7_2025-02,discrepancies75_m8_2025-01,discrepancies75_m9_2024-12,discrepancies75_m10_2024-11
0,Lanterns,22,5,0,0,5,0,0,8,0,0,6,4
1,Phones,21,6,0,0,8,2,1,9,1,0,7,1
2,SHS Entry-Level,35,12,0,0,9,0,0,12,1,0,9,2
3,SHS Entry-Level Upgrade,17,4,0,0,5,0,0,7,0,0,5,4
4,SHS with TV,32,10,0,1,9,2,0,12,0,3,16,5
5,SHS with TV Upgrade,25,4,0,0,3,0,0,11,1,10,9,13
6,SHS without TV,31,10,0,0,10,0,0,9,0,0,11,3


In [131]:
discrepancies_summary.to_csv("product_group.csv")